In [1]:
import pandas as pd
import os
from PIL import Image
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from pathlib import Path
import torch.nn.functional as F
import sys
from sklearn.metrics import accuracy_score, recall_score
import random
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from collections import Counter
import re
import requests

In [2]:
import cv2

In [3]:
url = "http://netdissect.csail.mit.edu/dissect/zoo/resnet-152-torch-places365.caffemodel"
save_path = "/path/to/your/directory/resnet-152-torch-places365.caffemodel"

def download_file(url, save_path):
    if os.path.exists(save_path):
        print("The file already exists, no need to download.")
    else:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(save_path, "wb") as file:
                for chunk in response.iter_content(1024):
                    file.write(chunk)
            print("File download successful, save path is:", save_path)
        else:
            print("File download failed, status code:", response.status_code)

files_to_download = [
    {
        "url": "http://netdissect.csail.mit.edu/dissect/zoo/resnet-152-torch-places365.caffemodel",
        "save_path": "../../models/resnet-152-torch-places365.caffemodel"
    },
    {
        "url": "http://netdissect.csail.mit.edu/dissect/zoo/resnet-152-torch-places365.prototxt",
        "save_path": "../../models/resnet-152-torch-places365.prototxt"
    }
]

for file in files_to_download:
    download_file(file["url"], file["save_path"])

The file already exists, no need to download.
The file already exists, no need to download.


In [4]:
# Define the path for the model architecture and weight files.
file_location_path = Path.cwd()
prototxt_path = '../../models/resnet-152-torch-places365.prototxt'
caffemodel_path = '../../models/resnet-152-torch-places365.caffemodel'
project_base_path = file_location_path.parent.parent
ns6_wiki_paths = project_base_path / 'data' / 'processed' / 'landscape_or_not'
image_folder = Path('/home/ubuntu/landscape-aesthetics')

results = []
# Load Caffe model.
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

category_mapping = {}
with open('../../data/external/Places365/categories_places365.txt', 'r') as file:
    for line in file:
        parts = line.strip().split()
        category_name = parts[0][3:]
        category_id = int(parts[1])
        category_mapping[category_id] = category_name

pattern = re.compile(r"^ns6_clean_(\d+)\.csv$") 

for file_name in tqdm(os.listdir(ns6_wiki_paths), desc="Processing files"):
    file_path = ns6_wiki_paths / file_name
    if file_path.is_file() and pattern.match(file_name):
        label_file_path = Path(file_name)
        labeled_csv_name = label_file_path.with_suffix('.csv')
        result = project_base_path / 'data' / 'processed' / 'landscape_type' / labeled_csv_name
        result.parent.mkdir(parents=True, exist_ok=True)

        results = []

        data = pd.read_csv(file_path, usecols=['image_path', 'prediction'])
        data = data[data['prediction'] == 1]
        image_paths = data['image_path'].tolist()
        for img_path in image_paths:
            image_path = image_folder / img_path
            image = cv2.imread(str(image_path))
            if image is None:
                continue 

            image = cv2.resize(image, (224, 224))
            blob = cv2.dnn.blobFromImage(
                image,
                scalefactor=1.0,
                size=(224, 224),
                mean=(123.675, 116.28, 103.53),
                swapRB=True,
                crop=False
            )

            net.setInput(blob)
            output = net.forward()

            top5_indices = output[0].argsort()[-5:][::-1]
            top5_probabilities = output[0][top5_indices]
            top5_categories = [category_mapping[idx] for idx in top5_indices]

            row = [img_path]
            for category, probability in zip(top5_categories, top5_probabilities):
                row.extend([category, probability])
            results.append(row)

        columns = ["filename", "prediction1", "probability1", "prediction2", "probability2",
                   "prediction3", "probability3", "prediction4", "probability4", 
                   "prediction5", "probability5"]

        df = pd.DataFrame(results, columns=columns)
        df.to_csv(result, index=False)

Processing files:   0%|                                 | 0/101 [00:00<?, ?it/s]

Processing files:   1%|▏               | 1/101 [3:58:08<396:54:59, 14288.99s/it]

Processing files:   2%|▎               | 2/101 [9:52:27<505:41:23, 18388.72s/it]

Processing files:   3%|▍              | 3/101 [18:02:08<637:13:04, 23408.00s/it]

Processing files:   4%|▌              | 4/101 [26:32:15<707:17:39, 26250.09s/it]

...

Processing files:  97%|█████████████▌| 98/101 [412:31:54<11:29:43, 13794.47s/it]

Processing files:  98%|██████████████▋| 99/101 [416:21:55<7:39:53, 13796.62s/it]

Processing files:  99%|█████████████▊| 100/101 [420:10:57<3:49:40, 13780.08s/it]

Processing files: 100%|████████████████| 101/101 [424:00:11<00:00, 13772.40s/it]

Processing files: 100%|████████████████| 101/101 [424:00:11<00:00, 15112.99s/it]